In [ ]:
import collections
import shelve
import sys
import tempfile

PY3 = (sys.version_info >= (3,0))

__version__ = '0.3.2'

def flatkeys(d, sep="/"):
    """
    Flatten a dictionary: build a new dictionary from a given one where all
    non-dict values are left untouched but nested ``dict``s are recursively
    merged in the new one with their keys prefixed by their parent key.

    >>> flatkeys({1: 42, 'foo': 12})
    {1: 42, 'foo': 12}
    >>> flatkeys({1: 42, 'foo': 12, 'bar': {'qux': True}})
    {1: 42, 'foo': 12, 'bar.qux': True}
    >>> flatkeys({1: {2: {3: 4}}})
    {'1.2.3': 4}
    >>> flatkeys({1: {2: {3: 4}, 5: 6}})
    {'1.2.3': 4, '1.5': 6}
    
    v0.1.0 by bfontaine, MIT license
    """
    flat = {}
    dicts = [("", d)]

    while dicts:
        prefix, d = dicts.pop()
        for k, v in d.items():
            k_s = str(k)
            if isinstance(v, collections.Mapping):
                dicts.append(("%s%s%s" % (prefix, k_s, sep), v))
            else:
                k_ = prefix + k_s if prefix else k
                flat[k_] = v
    return flat

# TODO: pb of not creating sub fdict with all required arguments, like delimiter or filename for sfdict
class fdict(dict):
    '''Flattened nested dict, all items are settable and gettable through ['item1']['item2'] standard form or ['item1/item2'] internal form.
    This allows to replace the internal dict with any on-disk storage system like a shelve's shelf (great for huge nested dicts that cannot fit into memory).
    Main limitation: an entry can be both a singleton and a nested fdict, and there is no way to tell what is what, no error will be shown, the singleton will always be returned.
    '''
    def __init__(self, d=None, rootpath='', delimiter='/', **kwargs):
        if d is not None:
            # Internal call, we get a subitem, we just create a new fdict with the same dictionary but with a restricted rootpath
            if rootpath:
                if isinstance(d, dict):
                    self.d = d
                else:
                    # sometimes (particularly extract(fullpath=True)) we get a list of tuples instead of a dict
                    self.d = dict(d)
            # Else it is not an internal call, the user supplied a dict to initialize the fdict, we have to flatten its keys
            elif isinstance(d, dict):
                self.d = flatkeys(d, sep=delimiter)
            # Else the user supplied another type of object, we try to convert to a dict and flatten it
            else:
                self.d = flatkeys(dict(d), sep=delimiter)
        else:
            self.d = dict()
        self.rootpath = rootpath
        self.delimiter = delimiter
        self.kwargs = kwargs
        self._py3compat()

    def _py3compat(self):
        if PY3:
            # Py3
            self._viewkeys = self.d.keys
            self._viewvalues = self.d.values
            self._viewitems = self.d.items
        else:
            # Py2
            if getattr(self.d, "viewvalues", None):
                # Py2.7
                self._viewkeys = self.d.viewkeys
                self._viewvalues = self.d.viewvalues
                self._viewitems = self.d.viewitems
            else:
                # Py2.6
                self._viewkeys = self.d.iterkeys
                self._viewvalues = self.d.itervalues
                self._viewitems = self.d.iteritems

    def _buildpath(self, key):
        return self.rootpath+self.delimiter+key if self.rootpath else key

    def __getitem__(self, key):
        # Node or leaf?
        if key in self.d: # Leaf: return the value
            return self.d.__getitem__(key)
        else: # Node: return a new full fdict based on the old one but with a different rootpath to limit the results by default (this is the magic that allows compatibility with the syntax d['item1']['item2'])
            return self.__class__(d=self.d, rootpath=self._buildpath(key), delimiter=self.delimiter, **self.kwargs)

    def __setitem__(self, key, value):
        #fullkey = self._buildpath(key)
        #if fullkey in self.d and :
        #    raise ValueError('Conflict detected: the following key is both a singleton and a nested dict: %s' % fullkey)

        if isinstance(value, dict):
            # if the value is a dict, flatten it recursively or drop if empty
            if not value:
                # User supplied an empty dict, the user wants to create a subdict, but it is not necessary here since nested dict are supported by default, just need to assign nested values
                return
            else:
                # else not empty dict, we flatten it and merge
                value = flatkeys({self.rootpath : value}, sep=self.delimiter)
                self.d.update(value)
        else:
            # if the value is not a dict, we just build the flattened key and store the value
            self.d.__setitem__(self._buildpath(key), value)

    def __delitem__(self, key):
        return self.d.__delitem__(self, self._buildpath(key))

    def __contains__(self, key):
        return self.d.__contains__(self, self._buildpath(key))

    #def addchild(self, key, value=None):
    #    self.d[self._buildpath(key)] = value

    def viewkeys(self, fullpath=False):
        if not self.rootpath:
            return self._viewkeys()
        else:
            pattern = self.rootpath+self.delimiter
            lpattern = len(pattern) if not fullpath else 0 # return the shortened path or fullpath?
            return (k[lpattern:] for k in self._viewkeys() if k.startswith(pattern))

    def viewitems(self, fullpath=False):
        # Filter items to keep only the ones below the rootpath level
        if not self.rootpath:
            return self._viewitems()
        else:
            pattern = self.rootpath+self.delimiter
            lpattern = len(pattern) if not fullpath else 0 # return the shortened path or fullpath?
            return ((k[lpattern:], v) for k,v in self._viewitems() if k.startswith(pattern))

    def viewvalues(self):
        if not self.rootpath:
            return self._viewvalues()
        else:
            pattern = self.rootpath+self.delimiter
            return (v for k,v in self._viewitems() if k.startswith(pattern))

    iterkeys = viewkeys
    itervalues = viewvalues
    iteritems = viewitems
    if PY3:
        keys = viewkeys
        values = viewvalues
        items = viewitems
    else:
        def keys(self, *args, **kwargs):
            return list(self.viewkeys(*args, **kwargs))
        def values(self, *args, **kwargs):
            return list(self.viewvalues(*args, **kwargs))
        def items(self, *args, **kwargs):
            return list(self.viewitems(*args, **kwargs))

    def update(self, d2):
        if isinstance(d2, self.__class__):
            # fdict supplied
            if self.rootpath == d2.rootpath:
                # we can directly merge because both contain the same rootpath and the full
                return self.d.update(d2.d)
            # TODO: what happens if d2 and self do not have same rootpath, or we do d['item'].update(d2) and d2 is an extract() of another fdict()? How to detect that case?
        else:
            d2 = flatkeys(d2, sep=self.delimiter)
            if self.rootpath:
                # There is a rootpath, so user is selecting a sub dict (eg, d['item1']), so we need to reconstruct d2 with the full key path before merging
                return self.d.update( ((self._buildpath(k), v) for k,v in d2.viewitems() ) )
            else:
                # No rootpath, we can update directly because both dicts are comparable
                return self.d.update(d2)

    def copy(self):
        return self.__class__(d=self.d.copy(), rootpath=self.rootpath, delimiter=self.delimiter, **self.kwargs)

    def __eq__(self, d2):
        if not self.rootpath:
            # No rootpath, we can just compare the dicts
            return (self.d == d2)
        else:
            # There is a rootpath, this is a subdict, so we have to filter the items we compare (else we will compare the full dict to d2, which is probably not what the user wants if he does d['item1'] == d2)
            for ((k, v), (k2, v2)) in zip(self.viewitems(), d2.viewitems()):
                if k != k2 or v != v2:
                    return False
            return True

    def __repr__(self):
        # Filter the items if there is a rootpath and return as a new fdict
        if self.rootpath:
            return repr(self.__class__(d=dict(self.items()), rootpath='', delimiter=self.delimiter, **self.kwargs))
        else:
            try:
                return self.d.__repr__()
            except AttributeError as exc:
                return repr(dict(self.items()))

    def __str__(self):
        if self.rootpath:
            return str(self.__class__(d=dict(self.items()), rootpath='', delimiter=self.delimiter, **self.kwargs))
        else:
            try:
                return self.d.__str__()
            except AttributeError as exc:
                return str(dict(self.items()))

    def to_dict(self):
        return dict(self.items())

    def extract(self, fullpath=True):
        # Return a new fdict shortened to only the currently subselected items, but instead of fdict, should also support sfdict or any child class
        # It was chosen to return a fdict still containing the full keys and not the shortened ones because else it becomes very difficult to merge fdicts
        # And also for subdicts (like sfdict) which might store in a file, so we don't want to start mixing up different paths in the same file, but we would like to extract to a fdict with same parameters as the original, so keeping full path is the only way to do so coherently.
        if fullpath:
            return self.__class__(d=self.items(fullpath=True), rootpath=self.rootpath, delimiter=self.delimiter, **self.kwargs)
        else:
            return self.__class__(d=self.items(fullpath=False), rootpath='', delimiter=self.delimiter) # , **self.kwargs)  # if not fullpath for keys, then we do not propagate kwargs because it might implicate propagating filename saving and mixing up keys. For fdict, this does not make a difference, but it might for subclassed dicts. Override this function if you want to ensure that an extract has all same parameters as original when fullpath=False in your subclassed dict.

    #def to_dict_nested(self):
    #    d2 = {}
    #    for k, v in self.viewitems():


class sfdict(fdict):
    '''A nested dict with flattened internal representation, combined with shelve to allow for efficient storage and memory allocation of huge nested dictionnaries.
    If you change leaf items (eg, list.append), do not forget to sync() to commit changes to disk and empty memory cache because else this class has no way to know if leaf items were changed!
    '''
    def __init__(self, *args, **kwargs):
        if not ('filename' in kwargs):
            self.filename = tempfile.NamedTemporaryFile(delete=False).name
        else:
            self.filename = kwargs['filename']
            #del kwargs['filename'] # do not del for auto management of internal sub calls to sfdict

        if 'autosync' in kwargs:
            self.autosync = kwargs['autosync']
            #del kwargs['autosync']
        else:
            self.autosync = True

        fdict.__init__(self, *args, **kwargs)
        self.d = shelve.open(filename=self.filename, flag='c', writeback=True)
        self._py3compat()

    def __setitem__(self, key, value):
        fdict.__setitem__(self, key, value)
        if self.autosync:
            self.sync()

    def get_filename(self):
        if self.filename:
            return self.filename
        else:
            return self.d.dict._datfile

    def sync(self):
        self.d.sync()

    def close(self):
        self.d.close()


In [ ]:
# Unit testing

# Test creation of just a nested dict, without anything else
a = fdict()
a['c']['b'] = set([1, 2])
assert a == {'c/b': set([1, 2])}

# Basic test
a = fdict()
a['a'] = {}
a['c']['b'] = set([1, 2])

assert a.keys() == ['c/b']
assert a.items() == [('c/b', set([1, 2]))]

# Copy test
acopy = a.copy()
assert acopy.items() == a.items()
assert acopy is not a

# Referencing into another variable of a nested item + check update of nested items
b = acopy['c']
assert b.items() == [('b', set([1, 2]))]
acopy['c'].update({'d': 3})
assert acopy == {'c/b': set([1, 2]), 'c/d': 3}
assert b == {'b': set([1, 2]), 'd': 3}

# Other tests
d = fdict()
d['b'] = {'a': 1}
d['c/b'] = set([2, 3, 5])
print('d', d)
assert d.to_dict() == {'c/b': set([2, 3, 5]), 'b': {'a': 1}}

a.update(d)
assert a.to_dict() == {'a': {}, 'c/b': set([2, 3, 5]), 'b': {'a': 1}}
assert a['c'].to_dict() == {'b': set([2, 3, 5])}

g = sfdict(filename='testshelf')
g['a'] = 3
g['b/c'] = set([1, 3, 4])
g['d'] = {}
assert g == {'a': 3, 'b/c': set([1, 3, 4]), 'd': {}}
assert g['b'].filename == g.filename # check that subdicts also share the same filename

h = sfdict(filename='testshelf')
assert h == g

m = {}
m['a'] = 1
m['b'] = {'c': 3, 'd': {'e': 5}}
m['f'] = set([1, 2, 5])
m2 = fdict(m)
assert dict(m2.items()) == flatkeys(m)

n = {}
n['b'] = {'d': {'f': 6}}
n['g'] = 7
m2.update(n)
assert m2 == {'a': 1, 'g': 7, 'b/d/f': 6, 'b/d/e': 5, 'f': set([1, 2, 5]), 'b/c': 3}

assert m2['b'].d == m2.d
assert m2['b'].extract().d == {'c': 3, 'd/e': 5, 'd/f': 6}

# Extract test
a10 = fdict()
a10['c/b/d'] = set([1, 2])
assert a10['c'].extract(fullpath=True).d == {'c/b/d': {1, 2}}
assert a10['c'].extract(fullpath=True) == {'b/d': {1, 2}}
assert a10['c'].extract(fullpath=False).d == {'b/d': {1, 2}}

In [ ]:
m2['h'] = {'i': {'j': 1}, 'k': set([5, 6, 7])}
m2

In [ ]:
print(d.viewvalues())

In [ ]:
a['a/b'] = 3
a['c']
a['c']['e'] = 1
a

In [ ]:
g = sfdict(filename='testshelf')
g['a'] = 3
g['b/c'] = set([1, 3, 4])
g['d'] = {}
g

In [ ]:
h = sfdict(filename='testshelf')
h

In [ ]:
### BENCHMARKS
try:
    _range = xrange
except NameError as exc:
    _range = range

def benchmark_set(dclass, breadth=5, depth=1000, args=None, kwargs=None):
    d = dclass(*args, **kwargs)
    di = d
    for i in _range(depth):
        for j in _range(breadth):
            d[str(j)] = j
        di = d[str(j)]
    return d

def benchmark_get(dclass, breadth=5, depth=1000, args=None, kwargs=None):
    d = benchmark_set(dclass, breadth=breadth, depth=depth, args=args, kwargs=kwargs)
    di = d
    x = None
    for i in _range(depth):
        for j in _range(breadth):
            x = d[str(j)]
        di = d[str(j)]
    return x
